In [1]:
# Setting up a custom stylesheet in IJulia
file = open("./../style.css") # A .css file in the same folder as this notebook file
styl = read(file, String) # Read the file
HTML("$styl") # Output as HTML

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.CodeMirror{\r\n        font-family: \"PT Mono\";\r\n        font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

## CUDA.jl (based on [CUDA.jl/ docs](https://cuda.juliagpu.org/stable/))

### Define a function, struct and  use it inside a kernel 


#### CPU implementation: 

In [2]:
using CUDA

struct Index
    ix::Integer
    iy::Integer
    iz::Integer
    
    function Index(indexes)
        new(indexes[1], indexes[2], indexes[3])
    end
end

# Function to map
function map_index(index, dim_img)
    onedim_index = (dim_img[1]*dim_img[2])*(index.iz - 1) + (dim_img[1])*(index.iy - 1) + index.ix
    return onedim_index
end

# CPU execution
dim_img = [1024, 1024, 100]
index = Index([10,2,3])

map_index(index, dim_img)

2098186

#### GPU implementation: 

In [3]:
# transform variables to cuda arrays
dim_img_gpu = CuArray(dim_img)
index_gpu = Index(CuArray([10,2,3]))
# try to access using 
# we obtain: 

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007fbc59178e70.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore /home/mvanzulli/.julia/packages/GPUArraysCore/rSIl2/src/GPUArraysCore.jl:81


Index(10, 2, 3)

This warning is a common pitfall for new incoming users, we really don't want to execute indexing in CPU, for instance for loops in cpu with CuArrays, because this will copy CuArray back to CPU, iterate and the copying it back to the GPU, what is really slow.  Also we can switch off indexing using `CUDA.allowscalar(fals)` to avoid indexing with CPU arrays. New versions doesn't allow to use indexing besides interactive execution modes. 

How to fix this? The answer is, that there is a conversion mechanism, which adapts objects into CUDA compatible bitstypes. It is based on the `Adapt.jl`
package and basic types like CuArray already participate in this mechanism. For custom types, we just need to add a conversion rule like so:

In [4]:
import Adapt 

function Adapt.adapt_structure(to, index::Index)
    ix = Adapt.adapt_strucutre(to, index.ix)
    iy = Adapt.adapt_strucutre(to, index.iy)
    iz = Adapt.adapt_strucutre(to, index.iz)
    return Index(ix, iy, iz)
end

#or 
# Alternatively instead of defining Adapt.adapt_structure explictly,
# we could have done
Adapt.@adapt_structure Index


### CuArrays

The CuArray type is an essential part of the toolchain. Is a resemble of Array. Primarily, it is used to manage GPU memory, and copy data from and back to the CPU. When we use `CuArray(variable)` then two operations are simultaneously being executed behind the scenes, memory allocation and copy into device.   

#### ax+b = y example 

In [5]:
# Compute the SAXPY product ax +b = c
# define constant variables 
const dim = 100_000_000
const a = pi 
# define a cu array
x = CUDA.ones(Float32, dim)
z = CuArray{Float32}(undef,dim)
# essential transformations with CuArrays
y = copy(x)
fill!(x,1)
# compute ax+b=z and we force the CPU to wait GPU finalization
CUDA.@sync z .= a.*x .+ y
# copy the result to CPU
z_cpu = Array(z)
# check results
import Test
Test.@test z_cpu == (a*ones(Float32, dim) + ones(Float32, dim))


Test Passed
  Expression: z_cpu == a * ones(Float32, dim) + ones(Float32, dim)
   Evaluated: Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593] == Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593]

### Operators bult-in kernels

There are some operators that we can use without writing an specific kernel for it, among them we dispose:


In [6]:
# cuArrays
α = CUDA.zeros(1024)
β = CUDA.ones(1024)
# CPUArrays
a_cpu = zeros(1024)
b_cpu = ones(1024)

result_cpu = a_cpu.^2 .+ sin.(b_cpu) 
result_gpu = α.^2 .+ sin.(β)

Test.@test abs(result_cpu[1] .- result_gpu[1]) < 1e-5 

Test Passed
  Expression: abs(result_cpu[1] .- result_gpu[1]) < 1.0e-5
   Evaluated: 3.159911643457747e-8 < 1.0e-5

When possible, these operations integrate with existing vendor libraries such as CUBLAS and CURAND. For example, multiplying matrices or generating random numbers will automatically dispatch to these high-quality libraries, if types are supported, and fall back to generic implementations otherwise.

### Vendor libraries 

For actually useful operations we can perform computations integrated with some NIVIDA's libraries that provide precompiled kernels for many operations. Among them we can use: 


In [7]:
#CUBLAS 
using LinearAlgebra
# perofrm A*B = Y
M = 500
N = 1_000
Y = CuArray{Float32}(undef, (M,1))
A = CUDA.rand(M,N)
B = CUDA.rand(N,1)
# use mul! implemented function 
mul!(Y, A, B)

500×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 230.63507
 239.95924
 239.69102
 241.92593
 241.84622
 241.20676
 246.0498
 232.88232
 248.21104
 245.14676
 252.61493
 249.52919
 240.1671
   ⋮
 240.44585
 247.27472
 247.73047
 249.997
 238.55968
 237.96555
 239.73299
 241.67624
 240.30276
 241.93796
 251.66449
 242.6794

In [8]:
# CUSOLVER 
qr(A)
# We have m

CUDA.CUSOLVER.CuQR{Float32, CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}} with factors Q and R:
Float32[-0.0084267855 0.009119287 … -0.014994912 -0.051567167; -0.028107788 -0.03481579 … -0.057056278 0.026883157; … ; -0.025895227 -0.046343774 … 0.029530665 0.017978534; -0.04862549 0.03881343 … 0.014941741 -0.03823671]
Float32[-13.179489 -9.867693 … -9.661791 -10.453778; 0.0 -8.631413 … -3.5047283 -2.944806; … ; 0.0 0.0 … 0.1839259 -0.021541096; 0.0 0.0 … -0.07726628 -0.47282785]

Writing wrappers for these libraries and integrating them with the relevant Julia interfaces or packages is not a difficult, but a very time consuming job.

To help with that, `CUDA.jl` also exposes all of the underlying C APIs, and makes them compatible with the `CuArray` type. For example, let's find the index of the smallest value using the cublasIsamin function:

In [9]:
# lets find the index of he salles value using culbasIsamin function f
out = Ref{Cint}()
CUBLAS.cublasIsamin_v2(CUBLAS.handle(), length(A), A, stride(A, 1), out)
out[]

433586

### Kernel programming 

But sometimes we need to declare and write our own kernels. Kernels are functions that are executed in a massively parallel fashion, and are launched by using the `@cuda` macro. The kernels are programmed with the SPMD simple program (the kernel) executed through Multiple Data. 

In [10]:
foo = CUDA.zeros(1024)

function kernel!(a)
    i = threadIdx().x
    a[i] += 1
    return nothing
end

@cuda threads = length(foo) kernel!(foo)
# copy to cpu
Test.@test Array(foo) == ones(1024)

Test Passed
  Expression: Array(foo) == ones(1024)
   Evaluated: Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] == [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

### y = ax+b example 

In [11]:
function axby(z, a, x, y)
    i = (blockIdx().x -1)* blockDim().x + threadIdx().x
    i <= length(z) && @inbounds z[i] = a * x[i] + y[i]
    return nothing
end

# define the GPU execution parameters
nthreads = CUDA.attribute(device(),
    CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)
# compute the number of blocks that overlay the dimenssion of arrays
nblocks = cld(dim, nthreads)

CUDA.@sync @cuda(
    threads = nthreads,
    blocks = nblocks,
    axby(z,a,x,y)
)
# copy the result to CPU
z_cpu = Array(z)
# check results
import Test
Test.@test z_cpu == (a*ones(Float32, dim) + ones(Float32, dim))



Test Passed
  Expression: z_cpu == a * ones(Float32, dim) + ones(Float32, dim)
   Evaluated: Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593] == Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593]

Some Julia features are not included when we are programming a kernel, such as: 

- allocate memory
- I/O is disallowed 
- badly-typed code will not compile 

Also we have to be aware that: 
- we need to **respect hardware limitations** 
- we need to efficiently use hardware resources **(occupancy)**
- **not every operation maps cleanly** on a scalar kernel

### Hardware limitations 

In [16]:
# lets check what happens if we have really big arrays 
x = CuArray(1:dim)
y = CuArray(1:dim)
z = similar(x)

nthreads = 2000
nblocks = cld(dim, nthreads)

CUDA.@sync @cuda(
    threads = nthreads,
    blocks = nblocks,
    axby(z,a,x,y)
)

LoadError: CUDA error: invalid argument (code 1, ERROR_INVALID_VALUE)

When we obtain this `ERROR_INVALID_VALUE` is because we have more threads than is alllwoed, we can insepct the maximum using 

```
nthreads = CUDA.attribute(device(),
    CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)
```

However using more complicated kernels local memory, shared memory may not allow to allocate all required memory. In order to solve that shortage we have an `Occupancy API`

### Occupancy API

We can figured it out the thread limit by compiling the kernel before lunching it to inspect its properties:

In [27]:
compiled_kernel = @cuda( 
    launch = false,
    axby(z,a,x,y)
)

println("Tha max number of threads that this kernel allows is $(CUDA.maxthreads(compiled_kernel))")
kernel_config = CUDA.launch_configuration(compiled_kernel.fun)

Tha max number of threads that this kernel allows is 1024


(blocks = 60, threads = 768)

In [ ]:
# so we now can set the threads and blocks using the occupancy API 

In [29]:
@show threads = min(length(x), kernel_config.threads )
@show blocks = cld(length(x), kernel_config.threads )

threads = min(length(x), kernel_config.threads) = 768
blocks = cld(length(x), kernel_config.threads) = 1303


1303

In [ ]:
# note that we can reuse the pre_complied kerne using 

### CUDA. BLAS solution

We can call CUBLAS module inside CUDA which provides all kinds of linear algebra functionalities for our CUDA apps: 

In [13]:
using CUDA, CUDA.CUBLAS 

x = CUDA.ones(Float32, dim)
y = CUDA.ones(Float32, dim)

# perform saxpy and overwrite y vector with axpy CUBLAS function
CUDA.@sync CUBLAS.axpy!(dim, a, x, y)

# copy results to cpu
y = Array(y)

Test.@test y == (a*ones(Float32, dim) + ones(Float32, dim))


Test Passed
  Expression: y == a * ones(Float32, dim) + ones(Float32, dim)
   Evaluated: Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593] == Float32[4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593  …  4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593, 4.141593]

### CUDA API wrappers 

For advanced use of the CUDA, you can use the driver API wrappers in CUDA.jl. Common operations include synchronizing the GPU, inspecting its properties, starting the profiler, etc. These operations are low-level, but for your convenience wrapped using high-level constructs. For example:

In [14]:
CUDA.@profile begin
    # code that runs under the profiler
end

# or execute this to show wich capabilitie
# s are avialable accordint to the device

for device in CUDA.devices()
    @show capability(device)
end

capability(device) = v"8.6.0"


┌ Warning: Calling CUDA.@profile only informs an external profiler to start.
│ The user is responsible for launching Julia under a CUDA profiler.
│ 
│ It is recommended to use Nsight Systems, which supports interactive profiling:
│ $ nsys launch julia
└ @ CUDA.Profile /home/mvanzulli/.julia/packages/CUDA/tTK8Y/lib/cudadrv/profile.jl:82
